In [21]:
import pandas as pd
from os import walk, listdir

important_filed = ['text', 'content-desc', 'resource-id', 'activity']

f = []


def get_migration_name(filename):

    migration_name = filename.replace('.json', '')
    if migration_name[0] =='a':
        return '', migration_name.split('-')[0], migration_name.split('-')[1]
    else:
        task_suffix = migration_name.split('-')[0][-1]
        if 'Expense' in migration_name:
            task = 'b6' + task_suffix
        elif 'Note' in migration_name:
            task = 'b7' + task_suffix
        else:
            task = 'b8' + task_suffix
        return migration_name.split('-')[0], migration_name.split('-')[1], task

def get_gt_from_dir(dir):

    for filename in listdir(dir):
        gt_df = pd.read_json(dir +'/'+ filename)
        try:
            gt_df = gt_df[important_filed]
        except:
            print()
        src, target, task = get_migration_name(filename)
        gt_df['src'] = src
        gt_df['target'] = target
        gt_df['task'] = task
        f.append(gt_df)

    return pd.concat(f)

atm_gt = get_gt_from_dir('atm_gt')
craft_gt = get_gt_from_dir('craft_gt')
total_gt = pd.concat([atm_gt, craft_gt])
col = ['src','target','task']
col.extend(important_filed)
total_gt = total_gt.reindex(columns= col)
total_gt.to_csv('gt_table.csv', index = False)